In [1]:
import os 
from typing import Any

import requests
from dotenv import load_dotenv



def load_env_key(key_to_load:str) -> Any:
    load_dotenv()
    return os.getenv(key_to_load)


def get_raw_json_response_from_db_id_notion(notion_api_key:str, 
                                            database_id:str,
                                            notion_version:str="2022-06-28") -> dict:
    
    headers = {
    "Authorization": f"Bearer {notion_api_key}",
    "Content-Type": "application/json",
    "Notion-Version": notion_version
    }
    
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    print("Requesting")
    response = requests.post(url, headers=headers)
    print("Requested")
    status_code = response.status_code
    
    assert status_code == 200 , f"Bad response ; {status_code}\nresponse : {response.text}"
    
    return response.json()
    

In [2]:
api_key:str = load_env_key(key_to_load="api_workspace")
database_id:str = load_env_key(key_to_load="database_id")

print(api_key)
print(database_id)


json_response = get_raw_json_response_from_db_id_notion(notion_api_key=api_key, database_id=database_id)

secret_FDl9bPlkaeOVM0sGrEjx0M3PCVWvUZrfdWhI7EQRMpx
441702ddc48a4b9daf5608b746020882
Requesting


In [ ]:
# Parse the page, get subpages
pages = json_response.get("results", [])
for page in pages:
    page_id = page['id']
    print(page.keys())
    print(page_id["properties"])
    page_title = page['properties']['Name']['title'][0]['plain_text']
    print(f"Page: {page_title}, ID: {page_id}")